<a href="https://colab.research.google.com/github/kavyajeetbora/modern_geospatial_stack/blob/master/notebooks/metro_accessibility.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install -q osmnx
!pip install -q pandana
!pip install -q pygaul
!pip install -q osmium
!pip install -q -U pyarrow
!pip install -q quackosm

In [13]:
import pygaul
import ee
import geemap
import geopandas as gpd
import pandas as pd
import osmium
import osmnx as ox
import shapely
import bz2
import os

ee.Authenticate()
ee.Initialize(project='kavyajeetbora-ee')

In [7]:
!wget https://download.geofabrik.de/asia/india/northern-zone-latest.osm.bz2 -O northern-zone-latest.osm.bz2

--2024-06-14 06:01:23--  https://download.geofabrik.de/asia/india/northern-zone-latest.osm.bz2
Resolving download.geofabrik.de (download.geofabrik.de)... 65.109.48.72, 65.109.50.43, 2a01:4f9:5a:25c3::2, ...
Connecting to download.geofabrik.de (download.geofabrik.de)|65.109.48.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 394812798 (377M) [application/x-bzip2]
Saving to: ‘northern-zone-latest.osm.bz2’

northern-zone-lates 100%[===================>] 376.52M  1.35MB/s    in 4m 23s  

2024-06-14 06:05:47 (1.43 MB/s) - ‘northern-zone-latest.osm.bz2’ saved [394812798/394812798]



In [14]:
admin = ee.FeatureCollection('FAO/GAUL_SIMPLIFIED_500m/2015/level2')

## Mumbai
Mumbai = admin.filter(ee.Filter.eq('ADM2_NAME', 'Mumbai city'))
Mumbai_suburban = admin.filter(ee.Filter.eq('ADM2_NAME', 'Mumbai Suburban'))
Mumbai = Mumbai_suburban.merge(Mumbai)

## Delhi
Delhi = admin.filter(ee.Filter.eq('ADM2_NAME', 'Delhi'))

## Bangalore
Bangalore = admin.filter(ee.Filter.eq('ADM2_NAME', 'Bangalore Urban'))

In [15]:
Map = geemap.Map()
Map.addLayer(ee_object=Mumbai, vis_params={'color':'red'}, name='Delhi')
Map.centerObject(Mumbai, zoom=10)
Map

Map(center=[19.120278006108997, 72.87471842263231], controls=(WidgetControl(options=['position', 'transparent_…

Converting `ee.Geometry` to `geopandas.GeoDataFrame`

In [22]:
def extract_polygons(gdf):
    polygons = []
    for geom_collection in gdf['geometry']:
        for geom in geom_collection.geoms:
            if isinstance(geom, shapely.geometry.polygon.Polygon):
                polygons.append(geom)


    geometry = shapely.unary_union(polygons)

    return geometry

In [27]:
## Delhi
Delhi = admin.filter(ee.Filter.eq('ADM2_NAME', 'Delhi'))
Bangalore = admin.filter(ee.Filter.eq('ADM2_NAME', 'Delhi'))
delhi_gdf = geemap.ee_to_gdf(Delhi)
bangalore_gdf = geemap.ee_to_gdf(Bangalore)

bangalore_gdf

,geometry,ADM0_CODE,ADM0_NAME,ADM1_CODE,ADM1_NAME,ADM2_CODE,ADM2_NAME,DISP_AREA,EXP2_YEAR,STATUS,STR2_YEAR,Shape_Area,Shape_Leng
0,"POLYGON ((76.83180 28.58173, 76.83853 28.55703...",115,India,1489,Delhi,17633,Delhi,NO,3000,Member State,1000,0.138635,2.064431


In [25]:
Mumbai_gdf = geemap.ee_to_gdf(Mumbai)

dfx = gpd.GeoDataFrame(extract_polygons(Mumbai_gdf).geoms)
dfx['area'] = dfx[0].apply(lambda x: x.area)
dfx = dfx.sort_values(by='area', ascending=False)
mumbai_geom = dfx[0].iloc[0]

Mumbai_gdf = Mumbai_gdf.drop(0,axis=0)
Mumbai_gdf.loc[1, 'geometry'] = mumbai_geom
Mumbai_gdf

,geometry,ADM0_CODE,ADM0_NAME,ADM1_CODE,ADM1_NAME,ADM2_CODE,ADM2_NAME,DISP_AREA,EXP2_YEAR,STATUS,STR2_YEAR,Shape_Area,Shape_Leng
1,"POLYGON ((72.79168 18.94520, 72.79168 18.94744...",115,India,1498,Maharashtra,70184,Mumbai city,NO,3000,Member State,1997,0.006605,0.794985


In [31]:
%%time
network = ox.graph.graph_from_polygon(bangalore_gdf.geometry.iloc[0], network_type='all', simplify=True, retain_all=False, truncate_by_edge=False, clean_periphery=None, custom_filter=None)

In [ ]:
network